In [1]:
import numpy as np
import torch as torch
import torch.nn as nn
from tqdm import tqdm

In [2]:
device = "cuda:1"

In [9]:
def trainer(model, gener, iters, opt, loss_fn, verbose= True, device= "cuda:1"):
    for i in tqdm(range(iters), desc = 'tqdm() Progress Bar'):
        (X, Y) = gener.com_get(64)
        preds= model(torch.tensor(X, device = device, dtype= torch.float32))
        loss= loss_fn(preds.transpose(1, 2), torch.tensor(Y, device = device))

        loss.backward()
        opt.step()
        opt.zero_grad()

        if i % 5000 == 0 and verbose:
            print("step %i: loss = %f"%(i, loss.detach().cpu().numpy()))

    print("final loss= %f"%(loss.detach().cpu().numpy()))
    

class sort():
    def __init__(self, len, ran):
        self.len = len
        self.ran= ran

    def get(self, num_swaps, b):
        sorted_seq = np.sort(np.random.choice(self.ran, [b, self.len]))
        seq = np.copy(sorted_seq)
        for _ in np.arange(num_swaps):
            i = np.random.randint(self.len-1, size= (b,))
            t= seq[np.arange(b), i]
            seq[np.arange(b), i] = seq[np.arange(b), i+1]
            seq[np.arange(b), i+1] = t
        seq = np.diag(np.ones(self.ran))[seq]

        return seq, sorted_seq
    
    def com_get(self, b):
        seq= np.random.choice(self.ran, [b, self.len])
        sorted_seq = np.sort(seq)
        seq = np.diag(np.ones(self.ran))[seq]
        
        return seq, sorted_seq
    
class sort_proj():
    def __init__(self, leng, ran):
        self.len = leng
        self.ran= ran
        self.proj = np.random.randn(ran, ran) / np.sqrt(ran)

    def get(self, num_swaps, b):
        sorted_seq = np.sort(np.random.choice(self.ran, [b, self.len]))
        seq = np.copy(sorted_seq)
        for _ in np.arange(num_swaps):
            i = np.random.randint(self.len-1, size= (b,))
            t= seq[np.arange(b), i]
            seq[np.arange(b), i] = seq[np.arange(b), i+1]
            seq[np.arange(b), i+1] = t
        seq = np.einsum('ijk,kl->ijl', np.diag(np.ones(self.ran))[seq], self.proj)

        return seq, sorted_seq
    
    def com_get(self, b):
        seq= np.random.choice(self.ran, [b, self.len])
        sorted_seq = np.sort(seq)
        seq = np.einsum('ijk,kl->ijl', np.diag(np.ones(self.ran))[seq], self.proj)
        
        return seq, sorted_seq
    

### Without positional encoding

In [10]:
class MultiHeadAttention_wthead_wtpos(nn.Module):
    def __init__(self, seq_len, input_dim, embed_dim, attn_dim, num_heads):
        super().__init__()
        assert embed_dim % num_heads == 0
        
        self.embed= nn.Linear(input_dim, embed_dim, bias = False)
        self.key = nn.Linear(embed_dim, attn_dim, bias = False)
        self.query = nn.Linear(embed_dim, attn_dim, bias = False)
        self.value = nn.Linear(embed_dim, attn_dim, bias = False)

        self.n_head = num_heads
        self.seq_len = seq_len
        self.input_dim= input_dim
        self.emd_dim = embed_dim
        self.attn_dim = attn_dim

    def forward(self, input):
        N, S, E = input.shape
        assert (S, E) == (self.seq_len, self.input_dim), "Wrong input!"

        X = self.embed(input)

        Q = torch.reshape(self.query(X), (N, self.seq_len, self.n_head, self.attn_dim//self.n_head)).transpose(1 , 2)
        K = torch.reshape(self.key(X), (N, self.seq_len, self.n_head, self.attn_dim//self.n_head)).transpose(1, 2)
        V = torch.reshape(self.value(X), (N, self.seq_len, self.n_head, self.attn_dim//self.n_head)).transpose(1, 2)
        scores= torch.matmul(Q, torch.transpose(K, 3, 2))/(self.attn_dim//self.n_head)**0.5

        Y1 = torch.matmul(scores, V).transpose(1, 2).reshape(N, self.seq_len, self.attn_dim)
        out_att = Y1 + input

        return out_att

In [11]:
for lr in np.logspace(-5, -2, 10):
    tran = MultiHeadAttention_wthead_wtpos(20, 200, 256, 200, 8)
    tran.to(device= device)
    gener= sort(20, 200)
    opt = torch.optim.Adam(tran.parameters(), lr= lr)
    loss_fn = nn.CrossEntropyLoss()
    print("learning rate= ", (lr))
    trainer(tran, gener, 5000, opt, loss_fn, verbose= False)

learning rate=  1e-05


tqdm() Progress Bar: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5000/5000 [00:13<00:00, 373.40it/s]


final loss= 5.092510
learning rate=  2.1544346900318823e-05


tqdm() Progress Bar: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5000/5000 [00:12<00:00, 394.45it/s]


final loss= 3.651713
learning rate=  4.641588833612782e-05


...

final loss= 3.583722
learning rate=  0.004641588833612777


tqdm() Progress Bar: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5000/5000 [00:12<00:00, 394.55it/s]


final loss= 3.668959
learning rate=  0.01


tqdm() Progress Bar: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5000/5000 [00:13<00:00, 381.98it/s]

final loss= 3.674258


In [12]:
lr = 5e-4
tran = MultiHeadAttention_wthead_wtpos(20, 200, 256, 200, 8)
tran.to(device= device)
gener= sort(20, 200)
opt = torch.optim.Adam(tran.parameters(), lr= lr)
loss_fn = nn.CrossEntropyLoss()
trainer(tran, gener, 250000, opt, loss_fn, verbose= True)

tqdm() Progress Bar:   0%|                                                                                                                                                 | 76/250000 [00:00<11:01, 378.03it/s]

step 0: loss = 5.250623


tqdm() Progress Bar:   2%|██▉                                                                                                                                            | 5060/250000 [00:12<11:04, 368.84it/s]

step 5000: loss = 3.531013


tqdm() Progress Bar:   4%|█████▋                                                                                                                                        | 10065/250000 [00:25<10:10, 392.77it/s]

...

step 95000: loss = 3.481259


tqdm() Progress Bar:  40%|████████████████████████████████████████████████████████▍                                                                                    | 100073/250000 [04:19<06:20, 393.84it/s]

step 100000: loss = 3.527942


tqdm() Progress Bar:  42%|███████████████████████████████████████████████████████████                                                                                  | 104766/250000 [04:31<06:16, 385.52it/s]


KeyboardInterrupt: 

In [14]:
(X, Y) = gener.get(400, 10000)
preds= tran(torch.tensor(X, device = device, dtype= torch.float32))
preds= torch.argmax(preds, dim = 2).detach().cpu().numpy()

print(np.sum(1*(Y == preds))/2e5)

0.086595
